# Strands Agent Deployment in Bedrock AgentCore Runtime with Observability

This workshop demonstrates how to deploy Strands Agents with Amazon Bedrock AgentCore Runtime, integrating multiple tools including MCP servers, code interpreters, and custom functions for comprehensive AI agent capabilities.

## Overview

In this lab, you will:
- Deploy a Strands Agent with multiple integrated tools
- Test the deployed agent with various scenarios
- Learn about observability and monitoring

## Prerequisites

Before starting this lab, ensure you have:
- AWS credentials configured (IAM role or environment variables)
- Required Python packages installed
- Basic understanding of Strands Agents and Bedrock AgentCore concepts

If you're not running in an environment with an IAM role assumed, set your AWS credentials as environment variables:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Install required packages for Strands Agents, MCP integration, and Bedrock AgentCore SDK:

In [ ]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore bedrock-agentcore-starter-toolkit

## What is Strands Agent with Bedrock AgentCore Runtime?

Strands Agents provide a powerful framework for building AI agents with multiple tool integrations. When deployed with Bedrock AgentCore Runtime, you get:

- **Scalable Deployment**: Managed infrastructure with auto-scaling capabilities
- **Secure Authentication**: Built-in security with Cognito integration
- **Observability**: CloudWatch integration for monitoring and debugging
- **Multi-Tool Integration**: Combine MCP servers in AgentCore Runtime, Code Interpreters, and Browser

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

@tool
def weather(city: str) -> str:
    """Get weather information for a city
    Args:
        city: City or location name
    """
    return f"Weather for {city}: Sunny, 35°C" # dummy result for demo purpose

# Create and test the comprehensive Strands Agent
agent = Agent(
    model=BedrockModel(model_id="us.amazon.nova-pro-v1:0"),
    system_prompt = """You are a helpful assistant that provides concise responses.
                    """,
    tools=[weather, calculator],
)

print("\n1. Testing custom weather tool:")
agent("How is the weather in HK?")

print("\n2. Testing calculator tool:")
agent("2+2=")

## Deploy Strands Agent to Bedrock AgentCore Runtime

Create a deployable version of our Strands Agent and deploy it as a managed service.
![bedrock-agentcore-runtime-launch](images/runtime-launch.png)

### Deployment Process:
1. Create Python file with agent configuration
2. Set up requirements.txt with dependencies
3. Configure AgentCore Runtime with authentication
4. Deploy using CodeBuild for containerization

### Step 1: Create Python File with Agent Configuration

Create the main Python file that defines our Strands Agent with all integrated tools. This file will be deployed to Bedrock AgentCore Runtime as a containerized service.

**Deployment Requirements:**
To deploy agents to AgentCore Runtime, we need to:
- Import the Runtime App: `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
- Initialize the App: `app = BedrockAgentCoreApp()`
- Decorate the invocation function with `@app.entrypoint`
- Let AgentCore Runtime control execution with `app.run()`

In [ ]:
%%writefile strands_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

@tool
def weather(city: str) -> str:
    """Get weather information for a city
    Args:
        city: City or location name
    """
    return f"Weather for {city}: Sunny, 35°C"


# Create and test the comprehensive Strands Agent
agent = Agent(
    model=BedrockModel(model_id="us.amazon.nova-pro-v1:0"),
    system_prompt = """You are a helpful assistant that provides concise responses.
                    """,
    tools=[weather, calculator],
)

@app.entrypoint
async def strands_agent_bedrock(payload, context):
    """
    Invoke the agent with a payload
    """
    print(f"Payload: {payload}")
    print(f"Context: {context}")
    user_input = payload.get("prompt", "No prompt found")
    reponse = agent(user_input)
    return response
    
    # Streaming Mode
    """
    stream = agent.stream_async(user_input)
    async for event in stream:
        if "data" in event:
            yield event
    """

if __name__ == "__main__":
    app.run()

## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

### Local Testing (Optional)

Before deploying to AgentCore Runtime, you can test the agent locally to verify functionality.

**Start Local Server:**
```bash
uv run strands_agent.py
```
or
```bash
python strands_agent.py
```
The server will start at `http://localhost:8080`

**Test with cURL:**
```bash
curl -X POST http://localhost:8080/invocations \
  -H "Content-Type: application/json" \
  -H "X-Amzn-Bedrock-AgentCore-Runtime-User-Id: 123" \
  -H "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id: 1234567890123456789012345678901234567890" \
  -d '{"prompt": "How is the weather in HK?"}'
```

### Step 2: Create Requirements File

Define the Python dependencies needed for our Strands Agent deployment.

**Key Dependencies:**
- **aws-opentelemetry-distro**: Required for AgentCore observability
- **strands-agents**: Core Strands framework
- **bedrock-agentcore**: Runtime integration

**Observability Integration:**
The `aws-opentelemetry-distro` library enables automatic instrumentation for monitoring and tracing. As documented in the [AgentCore Observability Guide](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability-configure.html), the containerized environment (such as docker) need to add the following command:

```dockerfile
CMD ["opentelemetry-instrument", "python", "main.py"]
```

This auto-instrumentation approach automatically adds the OpenTelemetry SDK to the Python path for comprehensive observability.

In [ ]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
bedrock-agentcore
bedrock-agentcore-starter-toolkit
aws-opentelemetry-distro>=0.10.0

### Observability Setup (Optional)

**CloudWatch APM Configuration:**
Enable CloudWatch APM → Transaction Search for comprehensive observability and monitoring of your deployed Strands Agent. In this lab, set X-Ray trace indexing to 100% for generating all trace summaries for end-to-end transaction analytics.
![bedrock-agentcore-observability-setup](images/observability-setup.png)

**Benefits:**
- **Performance Monitoring**: Request tracing and response time metrics
- **Error Tracking**: Comprehensive debugging capabilities
- **Usage Analytics**: Patterns and usage insights
- **Distributed Tracing**: End-to-end request flow visibility

### Step 3: Configure AgentCore Runtime

Set up the Bedrock AgentCore Runtime configuration with automatic resource creation.

**Generated Artifacts:**
This step creates essential deployment files:
- **Dockerfile**: Container configuration for the Strands Agent
- **.dockerignore**: list the excluded files when docker build
- **.bedrock_agentcore.yaml**: Runtime deployment configuration

Please note that when using the bedrock_agentcore_starter_toolkit to configure your agent, it takes care of the opentelemetry instrumentation. The generated Dockerfile will include:
```bash
CMD ["opentelemetry-instrument", "python", "runtime_agent_main.py"]
```

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3

region = boto3.session.Session().region_name
agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_getting_started"
)
response

### Step 4: Deploy to AgentCore Runtime

Launch the deployment process using AWS CodeBuild for containerization and deployment.

**Deployment Process:**
- Builds a containerized version of your Strands Agents
- Creates required AWS resources (ECR repository, IAM roles)
- Push container image to Amazon ECR
- Deploys to AgentCore Runtime as a managed, auto-scaling service

In [ ]:
launch_result = agentcore_runtime.launch()

### Step 5: Verify Deployment Status

Monitor the deployment progress and wait for the runtime to be ready:

In [ ]:
import time

print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

agent_runtime_id = launch_result.agent_id
agent_runtime_arn = launch_result.agent_arn
ecr_repo_name = launch_result.ecr_uri.split('/')[1]
codebuild_name = launch_result.codebuild_id.split(':')[0]
print(f"Strands AgentCore Runtime ID: {agent_runtime_id}")
print(f"Strands AgentCore Runtime ARN: {agent_runtime_arn}")
print(f"ECR Repo for Strands AgentCore Runtime: {ecr_repo_name}")
print(f"CodeBuild Project for Strands AgentCore Runtime: {codebuild_name}")

## Testing the Deployed Agent

Test the deployed Strands Agent by invoking it through the Bedrock AgentCore Runtime API with various prompts to verify all integrated tools are working correctly.

In [ ]:
import boto3
import json
import uuid

SESSION_ID = str(uuid.uuid4())

PROMPT = "How is the weather in HK?"

agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=boto3.session.Session().region_name
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_runtime_arn,
    qualifier="DEFAULT",
    runtimeSessionId=SESSION_ID, #Provide same session identifier across multiple requests to maintain conversation context, and with better traceability
    payload=json.dumps({"prompt": PROMPT})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                print(line)
else:
    events = []
    for event in boto3_response.get("response", []):
        print(event.decode('utf-8'))
        events.append(event)

### Testing Conversation History

Verify that the agent maintains conversation context across multiple requests using the same session ID:

In [ ]:
PROMPT = "What did I ask?"

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_runtime_arn,
    qualifier="DEFAULT",
    runtimeSessionId=SESSION_ID, #Provide same session identifier across multiple requests to maintain conversation context, and with better traceability
    payload=json.dumps({"prompt": PROMPT})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                print(line)
else:
    events = []
    for event in boto3_response.get("response", []):
        print(event.decode('utf-8'))
        events.append(event)

## Conversation History and Session Management

**Strands Agents Conversation Management:**
Strands Agents includes built-in conversation management with a default `SlidingWindowConversationManager` strategy. This automatically maintains conversation context within session, but not persistent across multiple sessions.

**Reference:** [Strands Agents Conversation Management](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/)

**AgentCore Runtime Session Isolation:**
- **Session Isolation**: Each user session is isolated and secure
- **Context Reuse**: Conversation history is maintained across multiple invocations
- **Session Duration**: Ephemeral sessions last up to 8 hours
- **Automatic Cleanup**: Sessions are automatically cleaned up after expiration

**Reference:** [AgentCore Runtime Sessions](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-sessions.html)


For persistent memory beyond session duration, integrate with **Bedrock AgentCore Memory** for long-term conversation storage and retrieval.

**Reference:** [AgentCore Memory: Add memory to your AI agent](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory.html)

## AgentCore Observability on Amazon CloudWatch

### What is Bedrock AgentCore Observability?

Amazon Bedrock AgentCore provides built-in observability through CloudWatch and X-Ray integration. This enables monitoring of agent performance, tracing request flows, and analyzing conversation patterns.

Key features:
- **Session Tracking**: Monitor individual conversations
- **Distributed Tracing**: Track requests across components
- **Performance Metrics**: Latency, throughput, and error rates
- **Span Analysis**: Detailed execution breakdown

Learn more: [AgentCore Observability Guide](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)

### Viewing the Main Dashboard

Access the CloudWatch console to view your AgentCore observability dashboard:
![observability-main-dashboard.png](images/observability-main-dashboard.png)

### Session Management

Click **DEFAULT** in `strands_claude_getting_started` to view session history. This shows our test session with two traces: "How is the weather in HK?" and "What did I ask?"
![observability-session-list.png](images/observability-session-list.png)

### Session Overview

Select a session to see metrics, trace timeline, and performance data:
![observability-session-overview.png](images/observability-session-overview.png)

### Trace Analysis

Click any trace to examine detailed execution steps:

**Trace Summary**:
![observability-trace-summary.png](images/observability-trace-summary.png)

**Span Details**:
![observability-trace-span-1.png](images/observability-trace-span-1.png)
![observability-trace-span-2.png](images/observability-trace-span-2.png)

## Resource Cleanup (Optional)

Clean up the deployed resources:

In [ ]:
import boto3
import os

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr',region_name=region)
codebuild_client = boto3.client('codebuild',region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    agentcore_control_client.delete_agent_runtime(agentRuntimeId=agent_runtime_id)
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_client.delete_repository(repositoryName=ecr_repo_name, force=True)
    print("✓ ECR repository deleted")

    print("Deleting CodeBuild Project...")
    codebuild_client.delete_project(name=codebuild_name)
    print("✓ CodeBuild Project deleted")

    print("Deleting Bedrock AgentCore configuration file...")
    os.remove(".bedrock_agentcore.yaml") 
    print("✓ .bedrock_agentcore.yaml deleted")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

## Conclusion

In this lab, you successfully:

- ✅ Deployed a Strands Agent to Bedrock AgentCore Runtime
- ✅ Configured observability and monitoring with Amazon CloudWatch
- ✅ Tested end-to-end agent functionality with invoking remote agent in AgentCore Runtime environment
- ✅ Explored session isolation and automatic cleanup capabilities

## Key Benefits of AgentCore Runtime for Strands Agents

- **Production Deployment**: Scalable, managed infrastructure for AI agents
- **Comprehensive Observability**: Built-in monitoring, tracing, and debugging capabilities
- **Session Management**: Automatic session isolation and cleaned up 
- **Enterprise Ready**: Security, reliability, and compliance features for production workloads
